In [1]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from utils.cuda_cluster import *
from utils.dataset import read_data, factorize_small_cardinality
from utils.util import get_day_phase, get_hours_from_midnight_of_current_day

import core.config as conf

/home/hyez/anaconda3/envs/dask-cudf/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34731 instead
  http_address["port"], self.http_server.port


In [2]:
client

Client Scheduler: tcp://127.0.0.1:39823 Dashboard: http://127.0.0.1:34731/status,Cluster Workers: 1 Cores: 1 Memory: 33.47 GB


## 1. Load data

In [3]:
# data_path = conf.raw_data_path + '*' # for all dadtaset
data_path = conf.raw_data_path + 'part-00175'
ori_df = read_data(data_path)

number of rows: 3033347


In [4]:
total_users = dask_cudf.concat([ori_df["engaged_with_user_id"], ori_df["enaging_user_id"]]).unique().compute()
ori_df, _ = factorize_small_cardinality(ori_df, 'tweet_id')
ori_df, _ = factorize_small_cardinality(ori_df, 'engaged_with_user_id', total_users, True)
ori_df, _ = factorize_small_cardinality(ori_df, 'enaging_user_id', total_users, True)
    

In [6]:
df = ori_df[["tweet_id_encode", "engaged_with_user_id_encode", "enaging_user_id_encode"]]
df['tweet_id_encode'] = df['tweet_id_encode'].astype(np.int32)
df['engaged_with_user_id_encode'] = df['engaged_with_user_id_encode'].astype(np.int32)
df['enaging_user_id_encode'] = df['enaging_user_id_encode'].astype(np.int32)

In [7]:
df.head()

,tweet_id_encode,engaged_with_user_id_encode,enaging_user_id_encode
id,,,
1,2214622,2586120,893844
2,2136815,3355961,3173376
3,1986297,1233947,951213
4,35685,3063535,2013995
5,1465500,2280964,3018294


In [8]:
df

,tweet_id_encode,engaged_with_user_id_encode,enaging_user_id_encode
npartitions=16,,,
1,int32,int32,int32
273141,...,...,...
...,...,...,...
2983130,...,...,...
3033347,...,...,...


## 2. Number of unique users, creators and engagers


In [9]:
unique_creators = df['engaged_with_user_id_encode'].unique().compute().to_array()
unique_engagers = df['enaging_user_id_encode'].unique().compute().to_array()
print(unique_creators)
print(unique_engagers)

[      4      10      14 ... 3511081 3511084 3511089]
[      0       1       2 ... 3511086 3511087 3511088]


In [10]:
unique_users = np.unique(np.append(unique_creators, unique_engagers))
print(unique_users)

[      0       1       2 ... 3511087 3511088 3511089]


## 3. Check the number of users that are both creators and engagers in our train set
